# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='/Users/lanjingyi/Desktop/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 黑天鵝到！　企業調整營運：延後開工17.8%、放無薪假15%

▲1111人力銀行職涯發展中心總經理李大華。（圖／記者余弦妙攝）
記者余弦妙／台北報導
受到新冠肺炎(COVID-19)疫情影響不少企業也跟著受到影響，1111人力銀行今（19）日公布最新調查顯示，不少企業因疫情影響紛紛調整營運措施，例如有17.8%延後開工、15.0%放無薪假、14.5%暫緩徵才、10.0%縮短營業時間/門市時間。
另外，而在調整內部措施方面，有44.1%提供口罩酒精等防疫用品、25.6%取消春酒、17.8%取消員工旅遊、11.1%實施彈性工時。
1111人力銀行職涯發展中心總經理李大華表示，中國已有多個城市宣布封城，打亂供應鏈，恐怕導致斷鏈危機，近來包括中國大陸、台灣、新加坡皆紛紛下修經濟成長率，加上疫情影響內需市場，中國大陸經濟雪上加霜，武漢肺炎恐成為今年影響中國經濟的第一隻「黑天鵝」，且根據IMF(國際貨幣基金)在1月發佈的最新「世界經濟展望」報告，也將今年全球經濟成長預期下修0.1%至3.3%。
李大華說，農曆年後通常是上班族轉職高峰期，求職者選擇多，應試機會大，唯今年受到武漢肺炎疫情衝擊，各行各業的業績表現不若往年暢旺，正職人力需求不顯著，造成徵才遞延「不是不找人，只是晚點找」的特殊現象。
----------------------------------------------------------------------

[1] 亞馬遜Ring啟動強制雙重驗證　防止駭客入侵

▲亞馬遜智慧居家安全公司Ring的監視器電鈴。（圖／達志影像／美聯社）
記者王曉敏／綜合報導
近年來安全性與隱私成為各大科技巨頭一大考驗，亞馬遜旗下家庭安全及智慧家居公司Ring昨（18）日宣布，即日起將強制要求用戶進行雙因素驗證（2 Factor Authentication，2FA）。每次登入後，都會以電子郵件或簡訊向用戶寄送一組一次性6位數代碼來驗證。
科技網站《Motherboard》去年12月披露，Ring並未採取任何基本措施來保護用戶的隱私，且雖然有為客戶提供雙因素驗證，但預設情況下並未要求用戶進行驗證。換句話說，若有心人士獲取了用

▲蘋果今推出新版本的watchOS。（圖／取自9TO5Mac）
記者謝仁傑／綜合報導
若要安裝更新軟體，Apple Watch需要至少有50％的電量，或放在充電器上，且需要放在iPhone的範圍內。
watchOS 6.1.3更新包括重要的錯誤修復，並解決冰島的Apple Watch用戶在不規則心律通知功能上出現的問題。
----------------------------------------------------------------------

[14] 防疫酒精來了！台糖產75%酒精全家今早十點開賣

▲台糖公司產製的75%平價防疫酒精，今（19）日上午十點，在FamilyMart全家便利商店正式開賣。（圖／記者唐詠絮攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情延燒，民眾對酒精的需求大幅增加，為解決一瓶難求現象，台糖公司產製的75%平價防疫酒精，今（19）日上午十點，在FamilyMart全家便利商店正式開賣，以滿足民眾防疫需求。
台糖強調，為了將有限資源更普及化，在全家便利商店開賣的75%防疫酒精採限量販售。每人限購1瓶，而且數量有限，賣完為止，建議不必搶購，另依政府相關法規18歲以下不得購買，請消費者注意。
台糖表示，目前臺灣在抗菌防疫上做得扎實，但還是不能鬆懈，特別是新冠病毒傳染途徑包括接觸傳染，所以勤洗手及消毒是阻擋病毒傳播很重要的方法，而濃度75%的酒精即可達到徹底清潔的功能。台糖配合防疫中心的需求，全力趕工生產75%平價防疫酒精，每瓶容量為350ml，售價45元。
----------------------------------------------------------------------

[15] 英格蘭銀行發行新版20英鎊塑膠鈔券　央行帶粉絲搶先看

▲ 英格蘭銀行正式推出新版20英鎊塑膠鈔券。（圖／翻攝自Facebook／中央銀行）
記者李瑞瑾／綜合報導
----------------------------------------------------------------------

[16] 韓元匯價刷新7年8個月以來新低　換新台幣5萬多一張愛寶樂園門票

▲ 韓元匯價走低。（圖／路透社）
記者李瑞瑾／台北報導
韓元匯價持續走低，根據台銀今（19）日上午牌告的現金賣出匯價，韓元來到0

▲南山人壽懲處公告引起各界議論紛紛。（圖／記者湯興漢攝）
記者楊絡懸／台北報導
南山人壽董事尹衍樑「公開打兒子」，要求記副董事長尹崇堯大過，引起軒然大波。總公司同仁見到懲處紛紛感到不可思議，「第一次看過懲處張貼在電梯口」；金管會保險局也議論「不符合公司治理」應為無效，要求南山提出相關說明。然而就在公告沒多久，南山人壽已悄悄撤下公文，並向金管會坦言「未經董事會通過，懲處無效」，劇情大轉折讓人疑惑。
南山人壽15日公告內容指出，由於南山人壽業績於2019年及2020年「連續二年」排名業界第3名，南山人壽董事尹衍樑要求，南山人壽副董事長尹崇堯及總經理許妙靜「各記一支大過，以儆效尤」。
這起懲處公文是保險業界首見公司高層「因業績不佳遭記大過」。值得一提的是，該公文以尹崇堯之名發布，內容卻有「尹衍樑董事要求」等字眼，可見是大股東父親發狠，親自懲罰副董事長兒子，然而疑點重重，例如該份公告不是董事長簽核，就算董事長不在，也應該要掛董事長的名字；再者，尹崇堯於2019年12月才上任副董事長，就要他對整年業績負責；而15日當天，南山內部到底發生什麼事，為何尹衍樑引爆怒火大罵公司經營階層？眾說紛紜。
據了解，南山人壽內部同仁在電梯口公布欄可以見到這份公文，也就是說，該懲戒公告以「昭告天下」方式向同仁「以儆效尤」，引起諸多遐想，也深怕下一波將會連累基層員工。南山工會當時也向《ETtoday新聞雲》記者表示，針對尹衍樑大股東直接介入保險公司經營管理及金融業經理人的專業判斷，恐嚴重影響保險公司的未來，將向金管會反映。
南山人壽業績連續兩年落後，位居第三，身為大股東的董事可能對業績感到不滿而要求懲處相關人員，但根據公司規定，高階人員懲處案件，必須到董事會提案並作出決議，而不是直接發布公告。金管會保險局聞訊更是「震怒」，官員認為，南山人壽此懲處案明顯未依公司章程及內控程序，不符合公司治理，要求南山人壽儘快說明，否則不排除再祭出新的「南山條款」。
不過，就在公告沒多久，南山人壽已悄悄撤下公文，就像「懲處沒有發生過」一樣；南山總公司內勤同仁向《ETtoday新聞雲》記者透露，當時看到懲處公文貼在電梯口，過了約10分鐘後，回去該處就發現「公文被撕掉了」。
金管會保險局表示，經過向南山人壽了解後，南山人壽坦言該公告「未依內部獎懲程序辦理」，也未經董事會通過，因此承認懲處無效，並予以撤銷；目前金管會已函請